In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table

In [2]:
engine = create_engine('sqlite:///batting.sql', echo=False)

In [3]:
engine.table_names()

['batting', 'bowling']

In [4]:
# Read the score card
df = pd.read_csv("../data/scoreboard-match-32-batting.csv")
dfb = pd.read_csv("../data/scoreboard-match-32-bowling.csv")

In [5]:
df

,BATSMEN,Unnamed: 1,R,B,4s,6s,SR
0,PP Shaw,c & b Gopal,47,25,4,4,188.00
1,C Munro,c †Buttler b Kulkarni,0,1,0,0,0.00
2,SS Iyer (c),c Tripathi b Unadkat,50,35,3,3,142.85
3,RR Pant †,c Stokes b Unadkat,69,29,7,5,237.93
4,GJ Maxwell,lbw b Archer,5,5,0,0,100.00
5,V Shankar,c Tripathi b Unadkat,17,6,2,1,283.33
6,LE Plunkett,not out,1,2,0,0,50.00
7,BATSMEN,NaN,R,B,4s,6s,SR
8,DJM Short,c Avesh Khan b Maxwell,44,25,2,4,176.00
9,JC Buttler †,st †Pant b Mishra,67,26,4,7,257.69


In [6]:
dfb

,Bowling,O,M,R,W,Econ,0s,4s,6s,WD,NB
0,DS Kulkarni,3,0,37,1,12.33,9,3,3,3,0
1,JC Archer,3.1,0,31,1,9.78,9,3,2,1,0
2,K Gowtham,2,0,27,0,13.50,2,4,1,0,0
3,JD Unadkat,4,0,46,3,11.50,7,4,2,0,0
4,S Gopal,2,0,26,1,13.00,6,0,4,0,0
5,BA Stokes,3,0,28,0,9.33,4,2,1,2,0
6,Bowling,O,M,R,W,Econ,0s,4s,6s,WD,NB
7,S Nadeem,1,0,13,0,13.00,1,1,1,0,0
8,TA Boult,3,0,26,2,8.66,5,3,0,3,0
9,Avesh Khan,2,0,36,0,18.00,2,2,4,0,0


In [7]:
# Renaming the Unnamed column
df.rename(columns={'Unnamed: 1':'FOW'}, inplace=True)
# Add innings column to the dataframe
df['Innings'] = 1 # Default 1

In [8]:
# Row indexes where the bowling and batting scorecard for different innings begin
bowler_indexes = df.loc[df['BATSMEN'] == 'Bowling'].index.tolist()
batsmen_indexes = df.loc[df['BATSMEN'] == 'BATSMEN'].index.tolist()
# First batting index as -1 because first innings starts at index 0
batsmen_indexes = [-1] + batsmen_indexes

In [10]:
# Break score card into innings
innings = [] # Row indexes of where the inning starts, where the bowling starts and where inning ends
for counter, ini in enumerate(batsmen_indexes):
    # Last inning ends at the end of the data frame so df.shape[0]
    if counter == len(batsmen_indexes) - 1:
        innings.append((batsmen_indexes[counter] + 1, bowler_indexes[counter] + 1, df.shape[0]))
    else:
        innings.append((batsmen_indexes[counter] + 1, bowler_indexes[counter] + 1, batsmen_indexes[counter + 1]))

IndexError: list index out of range

In [11]:
# Mapping to change the column names for bowling
bat_to_bowl = {'BATSMEN':'BOWLER', 'FOW':'O', 'R':'M', 'B':'R', 'M':'W', '4s':'Econ', '6s':'WD', 'SR':'NB'}
Batting = []
Bowling = []
# Change type of columns to make querying in sql possible
column_to_numeric_bat = ["R", "B", "M", "4s", "6s", "SR", 'Innings']
column_to_numeric_bowl = ["O", "M", "R", "W", "Econ", "WD", "NB", 'Innings']
# Make list of separate dataframes for bowling and batting of each innings
for counter, inning in enumerate(innings):
    # Replace deafult innings value by the real value
    df.loc[innings[counter][0]:innings[counter][1] - 1, 'Innings'].replace(1, counter + 1, inplace=True)
    # For batting
    bat = df[innings[counter][0]:innings[counter][1] - 1]
#     bat['Innings'].replace(1, counter + 1, inplace=True)
    bat[column_to_numeric_bat] =  bat[column_to_numeric_bat].apply(pd.to_numeric)
    Batting.append(bat)
    # For Bowling
    bowl = df[innings[counter][1]:innings[counter][2]]
#     bowl['Innings'].replace(1, counter + 1, inplace=True)    
    column_changed_bowl = bowl.rename(columns = bat_to_bowl)
    column_changed_bowl[column_to_numeric_bowl] =  column_changed_bowl[column_to_numeric_bowl].apply(pd.to_numeric)    
    Bowling.append(column_changed_bowl)

In [12]:
Batting[0]

IndexError: list index out of range

In [ ]:
Bowling[0]

In [13]:
# Check if the table exists, if not then populate it
if engine.dialect.has_table(engine.connect(), "batting"):
    pass
else:
    for x in range(len(Batting)):
        Batting[x].to_sql(name='batting', con=engine, if_exists='append')
        Bowling[x].to_sql(name='bowling', con=engine, if_exists='append')

In [14]:
engine.execute("SELECT BATSMEN FROM batting where R > 100").fetchall()

[('PP Shaw',), ('V Kohli (c)',)]

In [ ]:
def make_data(df):
    data = []
    for inning in df:
        for index, row in inning.iterrows():
            for column in inning.columns:
                data.append(tuple(("Row" + str(index), column, row[column])))
    return data

In [ ]:
data = []
data.append(make_data(Batting))
data.append(make_data(Bowling))

In [ ]:
data